In [5]:
import numpy as np
import os

In [3]:
# Paths to directories containing data
home_path = os.path.expanduser('~/')
path_67p = home_path+'Documents/year1/shape_modelling/67p/'

In [ ]:
# Read in magnitude data
readMikkoData = path_67p+'67p_'
f = open(readMikkoData, 'r')
lines = f.readlines()
for line in lines:
    data = line.split()
    